In [1]:
import os
import logging
from clickhouse_driver import Client
from dotenv import load_dotenv
from datetime import datetime, date

# Загрузка переменных окружения
load_dotenv()

# Настройка логирования
logging.basicConfig(
    level=logging.DEBUG,  # Установим уровень DEBUG для подробного логирования
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger()

# Глобальные переменные
database_name = os.getenv('CLICKHOUSE_DB', 'default')

# Подключение к ClickHouse
client = Client(
    host=os.getenv('CLICKHOUSE_HOST', '10.95.19.132'),
    user=os.getenv('CLICKHOUSE_USER', 'default'),
    password=os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi'),
    port=9000,
    secure=False,
    settings={'strings_encoding': 'utf-8'}
)


def fill_counters():
    """
    Заполняет счетчики sne, ppr и repair_days в таблице OlapCube_VNV на первую дату.
    """
    try:
        # Резюме операций
        total_processed = 0
        skipped_on_s = 0
        missing_vygruzka = 0
        repair_time_exceeded = []

        # Извлечь первую дату
        query_first_date = f"""
        SELECT MIN(Dates) AS first_date
        FROM {database_name}.OlapCube_VNV
        """
        first_date_result = client.execute(query_first_date)
        if not first_date_result or not first_date_result[0][0]:
            logger.error("Не удалось извлечь первую дату из OlapCube_VNV.")
            return
        first_date = first_date_result[0][0]
        logger.info(f"Первая дата в OlapCube_VNV: {first_date}")

        # Извлечь записи на первую дату из OlapCube_VNV вместе с RepairTime
        query_olap_data = f"""
        SELECT serialno, Status, daily_flight_hours, RepairTime
        FROM {database_name}.OlapCube_VNV
        WHERE Dates = '{first_date}'
        """
        olap_data = client.execute(query_olap_data)
        logger.info(f"Извлечено {len(olap_data)} записей на первую дату.")

        # Извлечь данные из Vygruzka
        query_vygruzka_data = f"""
        SELECT serialno, sne, ppr, removal_date
        FROM {database_name}.Vygruzka
        """
        vygruzka_data = client.execute(query_vygruzka_data)
        vygruzka_dict = {row[0]: row for row in vygruzka_data}
        logger.info(f"Извлечено {len(vygruzka_data)} записей из Vygruzka.")

        # Обработка данных
        for record in olap_data:
            serialno, status, daily_flight_hours, repair_time = record
            daily_flight_hours = daily_flight_hours or 0  # Защита от None

            # Пропуск serialno, начинающихся на S
            if serialno.startswith("S"):
                skipped_on_s += 1
                continue

            vygruzka_row = vygruzka_dict.get(serialno)
            if not vygruzka_row:
                missing_vygruzka += 1
                continue

            _, sne, ppr, removal_date = vygruzka_row
            sne = sne or 0
            ppr = ppr or 0

            if status == "Эксплуатация":
                new_sne = sne + daily_flight_hours
                new_ppr = ppr + daily_flight_hours
                update_query = f"""
                ALTER TABLE {database_name}.OlapCube_VNV
                UPDATE sne = {new_sne}, ppr = {new_ppr}
                WHERE serialno = '{serialno}' AND Dates = '{first_date}'
                """
                client.execute(update_query)

            elif status in ["Исправен", "Хранение"]:
                update_query = f"""
                ALTER TABLE {database_name}.OlapCube_VNV
                UPDATE sne = {sne}, ppr = {ppr}
                WHERE serialno = '{serialno}' AND Dates = '{first_date}'
                """
                client.execute(update_query)

            elif status == "Ремонт":
                repair_days = 0
                if removal_date:
                    try:
                        # Преобразуем removal_date и first_date в объекты date
                        if isinstance(removal_date, datetime):
                            removal_date_date = removal_date.date()
                        elif isinstance(removal_date, date):
                            removal_date_date = removal_date
                        elif isinstance(removal_date, str):
                            removal_date_date = datetime.strptime(removal_date, "%Y-%m-%d").date()
                        else:
                            logger.warning(f"Неизвестный тип removal_date для serialno {serialno}: {type(removal_date)}")
                            removal_date_date = None

                        if isinstance(first_date, datetime):
                            first_date_date = first_date.date()
                        elif isinstance(first_date, date):
                            first_date_date = first_date
                        elif isinstance(first_date, str):
                            first_date_date = datetime.strptime(first_date, "%Y-%m-%d").date()
                        else:
                            logger.warning(f"Неизвестный тип first_date: {type(first_date)}")
                            first_date_date = None

                        if removal_date_date and first_date_date:
                            delta = (first_date_date - removal_date_date).days
                            repair_days = max(delta, 0)  # Убедимся, что разница не отрицательная

                            # Логирование для отладки
                            logger.debug(f"Serialno: {serialno}, removal_date: {removal_date_date}, "
                                         f"first_date: {first_date_date}, delta: {delta}, repair_time: {repair_time}")

                            # Проверка на превышение RepairTime
                            if repair_time is not None:
                                # Преобразуем repair_time к float для универсальности
                                try:
                                    repair_time_numeric = float(repair_time)
                                except ValueError:
                                    logger.error(f"Некорректный формат RepairTime для serialno {serialno}: {repair_time}")
                                    repair_time_numeric = None

                                if repair_time_numeric is not None and repair_days >= repair_time_numeric:
                                    repair_days = int(repair_time_numeric)  # Приведение к int, если нужно
                                    repair_time_exceeded.append(serialno)
                                    logger.debug(f"repair_days ограничено RepairTime: {repair_days} для serialno {serialno}")
                        else:
                            logger.warning(f"Некорректные даты для serialno {serialno}: "
                                           f"removal_date_date={removal_date_date}, first_date_date={first_date_date}")
                    except Exception as date_error:
                        logger.error(f"Ошибка при обработке дат для serialno {serialno}: {date_error}", exc_info=True)
                        repair_days = 0

                # Обновление записи
                update_query = f"""
                ALTER TABLE {database_name}.OlapCube_VNV
                UPDATE sne = {sne}, ppr = {ppr}, repair_days = {repair_days}
                WHERE serialno = '{serialno}' AND Dates = '{first_date}'
                """
                client.execute(update_query)

            elif status == "Неактивно":
                update_query = f"""
                ALTER TABLE {database_name}.OlapCube_VNV
                UPDATE sne = 0, ppr = 0
                WHERE serialno = '{serialno}' AND Dates = '{first_date}'
                """
                client.execute(update_query)

            total_processed += 1

        # Итоговый лог
        logger.info(f"Обработано записей: {total_processed}")
        logger.info(f"Пропущено записей на S: {skipped_on_s}")
        logger.info(f"Пропущено записей из-за отсутствия данных в Vygruzka: {missing_vygruzka}")

        if repair_time_exceeded:
            logger.warning(f"Превышено RepairTime для следующих агрегатов: {', '.join(repair_time_exceeded)}")

        logger.info("Заполнение счетчиков завершено.")

    except Exception as e:
        logger.error(f"Ошибка при заполнении счетчиков: {e}", exc_info=True)


def main():
    fill_counters()


if __name__ == "__main__":
    main()


2025-03-13 14:55:51,029 - DEBUG - Connecting. Database: . User: default
2025-03-13 14:55:51,030 - DEBUG - Connecting to 10.95.19.132:9000
2025-03-13 14:55:51,101 - DEBUG - Connected to ClickHouse server version 24.10.1, revision: 54471
2025-03-13 14:55:51,102 - DEBUG - Query: 
        SELECT MIN(Dates) AS first_date
        FROM default.OlapCube_VNV
        
2025-03-13 14:55:51,102 - DEBUG - Block "" send time: 0.000027
2025-03-13 14:55:51,150 - DEBUG - Reading column first_date
2025-03-13 14:55:51,152 - DEBUG - Reading column host_name
2025-03-13 14:55:51,153 - DEBUG - Reading column current_time
2025-03-13 14:55:51,154 - DEBUG - /etc/timezone found, contents:
 Asia/Yekaterinburg

2025-03-13 14:55:51,156 - DEBUG - /etc/localtime found
2025-03-13 14:55:51,159 - DEBUG - 2 found:
 {'/etc/timezone': 'Asia/Yekaterinburg', '/etc/localtime is a symlink to': 'Asia/Yekaterinburg'}
2025-03-13 14:55:51,161 - DEBUG - Reading column thread_id
2025-03-13 14:55:51,161 - DEBUG - Reading column type
2